# Paso 1.
Preprocese los datos.
- Cree la columna 'Age' a partir de la columna 'Year'.
  Asuma que el año actual es 2021.
- Elimine las columnas 'Year' y 'Car_Name'.


In [1]:
import pandas as pd

In [2]:
ruta_train = "../files/input/train_data.csv.zip"
ruta_test = "../files/input/test_data.csv.zip"
df_train = pd.read_csv(ruta_train, compression='zip')
df_test = pd.read_csv(ruta_test, compression='zip')

df_train.head()
#df_test.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,jazz,2016,7.40,8.500,15059,Petrol,Dealer,Automatic,0
1,i10,2013,4.00,4.600,30000,Petrol,Dealer,Manual,0
2,TVS Apache RTR 180,2011,0.50,0.826,6000,Petrol,Individual,Manual,0
3,eon,2016,3.15,4.430,15000,Petrol,Dealer,Manual,0
4,Royal Enfield Thunder 350,2013,1.25,1.500,15000,Petrol,Individual,Manual,0


In [3]:
df_train['Age'] = 2021 - df_train['Year']
df_test['Age'] = 2021 - df_test['Year']

df_train.head()
#df_test.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,jazz,2016,7.40,8.500,15059,Petrol,Dealer,Automatic,0,5
1,i10,2013,4.00,4.600,30000,Petrol,Dealer,Manual,0,8
2,TVS Apache RTR 180,2011,0.50,0.826,6000,Petrol,Individual,Manual,0,10
3,eon,2016,3.15,4.430,15000,Petrol,Dealer,Manual,0,5
4,Royal Enfield Thunder 350,2013,1.25,1.500,15000,Petrol,Individual,Manual,0,8


In [4]:
df_train = df_train.drop(['Year', 'Car_Name'], axis=1)
df_test = df_test.drop(['Year', 'Car_Name'], axis=1)

df_train.head()

,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,7.40,8.500,15059,Petrol,Dealer,Automatic,0,5
1,4.00,4.600,30000,Petrol,Dealer,Manual,0,8
2,0.50,0.826,6000,Petrol,Individual,Manual,0,10
3,3.15,4.430,15000,Petrol,Dealer,Manual,0,5
4,1.25,1.500,15000,Petrol,Individual,Manual,0,8


In [5]:
#df_test.head()

In [6]:
# print(df_train.shape)
# print(df_test.shape)

In [7]:
# Ni train ni test tienes valores nulos

#df_train[df_train.duplicated(keep='first')] # Solo un duplicado

In [8]:
# df_test no tiene duplicados

# Paso 2.
Divida los datasets en x_train, y_train, x_test, y_test.

In [9]:
X_train = df_train.drop(['Present_Price'], axis=1)
y_train = df_train['Present_Price']

X_test = df_test.drop(['Present_Price'], axis=1)
y_test = df_test['Present_Price']

#X_train.head()

# Paso 3.
Cree un pipeline para el modelo de clasificación. Este pipeline debe
contener las siguientes capas:
- Transforma las variables categoricas usando el método
  one-hot-encoding.
- Escala las variables numéricas al intervalo [0, 1].
- Selecciona las K mejores entradas.
- Ajusta un modelo de regresion lineal.

In [10]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

In [11]:
categorical = ['Fuel_Type','Selling_type','Transmission']
numerical = [col for col in X_train.columns if col not in categorical]

preprocessor = ColumnTransformer(
    transformers=[
        ('categories', OneHotEncoder(handle_unknown="ignore", dtype=int), categorical),
        ('scaler', MinMaxScaler(),numerical)
        ],
    #remainder='passthrough' # Mantiene el resto de las columnas sin modificar
)

k = SelectKBest(score_func=f_regression)
lr = LinearRegression(n_jobs=-1)

steps = [
    ('preprocessor',  preprocessor),
    ('selectkbest', k),
    ('classifier', lr)
]

pipeline = Pipeline(steps)

In [12]:
#X_train.shape[1]

# Paso 4.
Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada. Use el error medio absoluto para medir el desempeño modelo.

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:

param_grid = {
    'selectkbest__k': range(1, 25),
    'classifier__fit_intercept':[True, False],
    'classifier__positive':[True,False],
    'classifier__copy_X':[True,False]
}


model = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=10,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    refit=True
)

model.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categories',
                                                                         OneHotEncoder(dtype=<class 'int'>,
                                                                                       handle_unknown='ignore'),
                                                                         ['Fuel_Type',
                                                                          'Selling_type',
                                                                          'Transmission']),
                                                                        ('scaler',
                                                                         MinMaxScaler(),
                                                                         ['Selling_Price',
                                                                          'Driven_kms',
                                                                          'Owner',
                                                                          'Age'])])),
                                       ('selectkbest',
                                        SelectKBest(score_func=<function f_regression at 0x000001F9DA75E950>)),
                                       ('classifier',
                                        LinearRegression(n_jobs=-1))]),
             n_jobs=-1,
             param_grid={'classifier__copy_X': [True, False],
                         'classifier__fit_intercept': [True, False],
                         'classifier__positive': [True, False],
                         'selectkbest__k': range(1, 25)},
             scoring='neg_mean_absolute_error')

In [15]:
print('Mejores parámetros: ', model.best_params_)
print()
print("Score en train: ", model.score(X_train, y_train))
print("Score en test: ", model.score(X_test, y_test))

Mejores parámetros:  {'classifier__copy_X': True, 'classifier__fit_intercept': True, 'classifier__positive': True, 'selectkbest__k': 11}

Score en train:  -1.6214879816158285
Score en test:  -2.473619914416376


# Paso 5.
Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
- Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [16]:
import pickle
import gzip
import os

In [17]:
os.makedirs('../files/models', exist_ok=True)

with gzip.open("../files/models/model.pkl.gz","wb") as file:
    pickle.dump(model, file)

# Paso 6.
Calcule las metricas r2, error cuadratico medio, y error absoluto medio para los conjuntos de entrenamiento y prueba. 

Guardelas en el archivo files/output/metrics.json. 

- Cada fila del archivo es un diccionario con las metricas de un modelo. 
- Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. 

Por ejemplo:

{'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}

{'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

In [18]:
import json

from sklearn.metrics import (
    r2_score, 
    mean_squared_error, 
    mean_absolute_error,
    median_absolute_error
)

In [19]:
def calcular_metricas(modelo, X_dataset, y_dataset, tipo_dataset):
    y_pred = modelo.predict(X_dataset)
    metrics = {
    "type": "metrics", # Para el test
    'dataset': tipo_dataset,
    'r2': r2_score(y_dataset, y_pred),
    'mse': mean_squared_error(y_dataset, y_pred),
    #'mad': mean_absolute_error(y_dataset, y_pred), #Ensayar con mediana como alternativa a la media
    'mad': median_absolute_error(y_dataset, y_pred),
    }
    return metrics

In [20]:
metricas_train = calcular_metricas(modelo= model, X_dataset=X_train, y_dataset=y_train, tipo_dataset='train')
metricas_test = calcular_metricas(modelo= model, X_dataset=X_test, y_dataset=y_test, tipo_dataset='test')

metricas = [metricas_train, metricas_test]

os.makedirs('../files/output', exist_ok=True)

with open('../files/output/metrics.json', 'w') as file:
    for metrica in metricas:
        file.write(json.dumps(metrica) + '\n')